In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp agents/es_agent

In [ ]:
#| export
from llama_index.core.workflow import Event,StartEvent,StopEvent,Workflow,step, Context

from typing import Any
from typing import List
from llama_index.core.memory import ChatMemoryBuffer
from pydantic import BaseModel, Field

from multi_agent_llms.utils.opensearch_utils import recursive_literal_eval
from multi_agent_llms.utils.agent_prompts import QueryDetails
from multi_agent_llms.utils.agent_prompts import ResponseSynthesizer
import pandas as pd
from IPython.display import display
import mlflow
import json


In [ ]:
#| export
class UserState(BaseModel):
    user_query:str = Field(description='User query')
    schema_selector_error_lists:list=Field(default=[])
    query_execution_error_lists: list=Field(default=[])
    validation_error_list:list=Field(default=[])
    error_lists:list=Field(default=[])
    schema_selector_response:object=Field(default=None)
    query_generator_response:object = Field(default=None)
    query_validator_response:object = Field(default=None)
    query_response_synthesis:object = Field(default=None)
    query_validation_run_count:int=0
    query_run_count:int=0
    es_response:dict=dict()
    exited_with_error:bool=False
    exited_stage:str=''

In [ ]:
#| export

class QueryState(BaseModel):
    query:str|dict = Field(description='ES Query')
    query_creation_stage:str = Field(default=None,description='Where the query was created')
    query_creation_timestamp:str = Field(default=str(pd.Timestamp.utcnow()),description='ES Query creation timestamp')
    rejection_timestamp:str = Field(default=None,description='Rejection timestamp')
    cot:List[str] = Field(default=[], description='ES Query')
    rejection_reasons:List[str] = Field(default=[], description='Reasons for rejection')
    validation_score:float = Field(default=None,description='validation score')
    status:bool = Field(default=False,description='Status of query execution or validation')
    rejection_stage:str = Field(default='',description='Stage where the query was rejected')
    query_id:int = Field(default=None,description='Query Number')
    run_id:str = Field(default='default',description='Run Id')



In [ ]:
#| export
class MLFlowLamaIndexLogger(object):
    def __init__(self, session_id, experiment_name:str):
        self.session_id = session_id
        self.run_id = self.get_run_id()
        self.experiment_name = experiment_name
    
    def get_run_id(self):
        if self.session_id is not None:
            return f'{self.session_id}_{str(pd.Timestamp.now())}'
        else:
            return f'default_{str(pd.Timestamp.now())}'
    
    def start_mlflow_logging(self):
        mlflow.set_experiment(self.experiment_name)
        mlflow.start_run()
        mlflow.set_tag("mlflow.runName", self.run_id) 
        mlflow.llama_index.autolog(disable=False)
    
    def end_mlflow_logging(self):
        mlflow.llama_index.autolog(disable=True)
    
    def log_json_artifact(self, file_name:str, data):    
        # Optionally, you can log it as an artifact (if you prefer saving it as a file)
        with open(f"{file_name}.json", "w") as f:
            json.dump(data, f)
        mlflow.log_artifact(f"{file_name}.json")
        


In [ ]:
#| export
class MlFlowLamaIndexESAgentLoggerDataPrepare(object):
    def __init__(self, run_id):
        self.run_id:str = run_id
        self.current_query_state:QueryState = None
        self.query_list:List[QueryState] = []
        
    def initialize_query_state(self, data, stage):
        self.current_query_state= QueryState(query=data.query, 
                                             cot=data.cot, 
                                            query_creation_stage=stage, 
                                            query_creation_timestamp=str(pd.Timestamp.utcnow())
                                            )

        
    
    def update_variables_on_rejection(self, stage:str, rejection_reasons:List[str]):
        self.current_query_state.rejection_stage = stage
        self.current_query_state.run_id = self.run_id
        self.current_query_state.status = False
        self.current_query_state.rejection_reasons = rejection_reasons
        self.current_query_state.query_id = len(self.query_list)+1
        self.current_query_state.rejection_timestamp = str(pd.Timestamp.utcnow())

        self.query_list.append(self.current_query_state.model_dump())
        self.current_query_state = None 
    
    def update_variables_on_sucess(self):
        self.current_query_state.status = True
        self.current_query_state.run_id = self.run_id
        self.current_query_state.query_id = len(self.query_list)+1
        self.query_list.append(self.current_query_state.model_dump())
        self.current_query_state = None 




In [ ]:
#| export
class EstablishEnvEvent(Event):
    pass

class SchemaSelectorEvent(Event):
    pass

class QueryGeneratorEvent(Event):
    pass

class QueryValidatorEvent(Event):
    pass

class QueryRunCorrectionEvent(Event):
    pass

class QueryValidationCorrectionEvent(Event):
    pass

class QueryRunEvent(Event):
    pass

class ResultInferenceEvent(Event):
    pass

class ContextInferenceEvent(Event):
    """
    Event to be triggred when the solution is found in the context and doesnot need to query ES
    """
    pass

class ErrorEvent(Event):
    error_stage:str

class ErrorCompleteEvent(Event):
    pass


In [ ]:
#| export
class ElastiSearchcAgent(Workflow):
    """
    Multi-agent to query ES instance
    1. The agent will create and run the query against an es instance
    2. Actor-critic paradigm utilzed to improve system relaiability. LLM as judge
    3. LLM used generate inferences from es results
    """
    def __init__(self, llm:object, agent_dict:dict, schema_dict:dict, opensearch_client:object,city=str,
                 max_errors:int=1, session_id:str=None,*args: Any, **kwargs: Any)->None:
        
        super().__init__(*args, **kwargs)
        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.llm = llm
        self.agent_dict=agent_dict
        self.error_dict = dict()
        self.max_errors = max_errors
        self.schema_dict = schema_dict
        self.opensearch_client = opensearch_client
        self.city=city
        self.session_id = session_id

        

    def run_agent(self, agent, agent_prompt):
        sllm = self.llm.as_structured_llm(agent.output_class)
        response = sllm.complete(agent_prompt)
        response = recursive_literal_eval(response.text)
        return agent.output_class(**response)

    def handle_error(self, stage, error):
        print('*'*50)
        print('Ecountered error in ', stage)
        print(error)
        self.user_state.schema_selector_error_lists.append(error.__str__)

    @step
    async def handle_establish_env(self, ctx: Context,  ev:StartEvent) -> EstablishEnvEvent:
        """
        Establish user state and execution env
        """
        self.logger = MLFlowLamaIndexLogger(session_id=self.session_id, experiment_name="ES Agent Workflow Tutorial")
        self.logger.start_mlflow_logging()
        
        self.logger_data_prepare = MlFlowLamaIndexESAgentLoggerDataPrepare(run_id=self.logger.run_id)
        mlflow.log_param('user_query', ev.query)
        assert self.llm is not None

        # hold the user state for the current querx
        self.user_state = UserState(user_query=ev.query)
        return EstablishEnvEvent()

    @step
    async def handle_schema_selector_event(self, ctx: Context,  ev:EstablishEnvEvent)->SchemaSelectorEvent|ErrorEvent:
        """
        Select the indexes to answer the user query
        Args:
            ctx (Context): _description_
            ev (EstablishEnvEvent): _description_

        Returns:
            SchemaSelectorEvent: when we have sucessfully selected relevant events and forward to query generator
            ErrorEvent: in case of an error
        """
        try:
            agent = self.agent_dict['schema_selector']
            agent_prompt = agent.agent_prompt.format(query_str=self.user_state.user_query,
                                                    schema_str=self.schema_dict
                                                    )
            self.user_state.schema_selector_response = self.run_agent(agent=agent, agent_prompt=agent_prompt)
            selected_schemas = [x.variable_name for x in self.user_state.schema_selector_response.indexes]
            mlflow.log_param(key='selected schemas', value=selected_schemas)
            return SchemaSelectorEvent()
        except Exception as e:
            self.handle_error('schema selector', e)
            return ErrorEvent(error_stage='schema selection')

    @step
    async def handle_query_generator(self, ctx: Context,  ev:SchemaSelectorEvent)->QueryGeneratorEvent|ErrorEvent|ContextInferenceEvent:
        """
        Use the schema selectors to generate es query
        Args:
            ctx (Context): _description_
            ev (SchemaSelectorEvent): _description_

        Returns:
            QueryGeneratorEvent: successful generation of query and forward it to query validation
            ErrorEvent: in case of an error
            ContextInferenceEvent: can be answered without any es query
        """
        schema_response = self.user_state.schema_selector_response
        try:
            agent = self.agent_dict['query_generator']
            agent_prompt = agent.agent_prompt.format(query_str=self.user_state.user_query,
                                                        schema_str=schema_response,
                                                        input_str=QueryDetails.model_json_schema()
                                                            )
            response = self.run_agent(agent=agent, agent_prompt=agent_prompt)
            self.user_state.query_generator_response = response

            if len(self.user_state.query_generator_response.context_solution)>0 and len(self.user_state.query_generator_response.query)==0:
                self.query_state = QueryState(query='Context Inference', cot=response.cot)
                return ContextInferenceEvent()
            
            elif len(self.user_state.query_generator_response.query)==0 and len(self.user_state.query_generator_response.context_solution)==0:
                return ErrorEvent(error_stage='query generator')
        
            self.logger_data_prepare.initialize_query_state(data=response, stage='query generator')
            
            return QueryGeneratorEvent()
        except Exception as e:
            self.handle_error('query generator', e)
            return ErrorEvent(error_stage='query generator')
    
    @step
    async def handle_context_inference_generator(self, ctx: Context,  ev:ContextInferenceEvent)->ResultInferenceEvent|ErrorEvent:
        """
        Handle when query can be answered without es help
        Args:
            ctx (Context): _description_
            ev (ContextInferenceEvent): _description_

        Returns:
            ResultInferenceEvent: update the resposne and finish sucessfull execution
            ErrorEvent: in case of error
        """
        try:
            response = ResponseSynthesizer(user_query=self.user_state.user_query,
                                           response=self.user_state.query_generator_response.context_solution
                                           )
            self.user_state.query_response_synthesis = response
            return ResultInferenceEvent()
        except Exception as e:
            self.handle_error('Context Inference', e)
            return ErrorEvent(error_stage='Context Inference event')

    @step
    async def handle_query_validator_event(self, ctx: Context,  ev:QueryGeneratorEvent)->QueryValidatorEvent|ErrorEvent|QueryValidationCorrectionEvent:
        """
        Symenatic validaton of the query and the user request
        LLM as a judge with the critic model
        Args:
            ctx (Context): _description_
            ev (QueryGeneratorEvent): _description_

        Returns:
            QueryValidatorEvent: sucessful validation forward to runner
            ErrorEvent: in case of error
            QueryValidationCorrectionEvent: correct the query incase the threshold is not statisfied
        """
        try:
            print('*'*50)
            print('Running Query evalution no:', self.user_state.query_validation_run_count)
            display(self.user_state.query_generator_response.query)

            agent = self.agent_dict['query_validator']
            agent_prompt = agent.agent_prompt.format(query_str=self.user_state.user_query,
                                                        schema_str=self.user_state.schema_selector_response.indexes,
                                                        input_str=QueryDetails.model_json_schema(),
                                                        es_query=self.user_state.query_generator_response.query
                                                            )
            response = self.run_agent(agent=agent, agent_prompt=agent_prompt)
            self.user_state.query_validator_response =  response
            self.logger_data_prepare.current_query_state.validation_score = response.query_score
            
            assert self.user_state.query_validation_run_count < self.max_errors, "maximum iterations for the query validator reached"
            if response.query_score < .9:
                self.logger_data_prepare.update_variables_on_rejection(stage='query validation', 
                                                                       rejection_reasons=response.reason
                                                                       )
                
                self.user_state.query_validation_run_count += 1
                self.user_state.validation_error_list.append(self.user_state.query_validator_response.reason)
                print('Failed Query evalution :', self.user_state.query_validator_response.reason)
                return QueryValidationCorrectionEvent()
            
            self.user_state.query_validation_run_count = 0
            return QueryValidatorEvent()
        
        except Exception as e:
            self.logger_data_prepare.update_variables_on_rejection(stage='query validation', 
                                                                       rejection_reasons=[e.__str__]
                                                                       )
            self.handle_error('Query Validations', e)
            return ErrorEvent(error_stage='query validation')
    
    @step
    async def handle_query_run_event(self, ctx: Context,  ev:QueryValidatorEvent)->QueryRunEvent|QueryRunCorrectionEvent|ErrorEvent:
        """
        Execute query against ES instance

        Args:
            ctx (Context): _description_
            ev (QueryValidatorEvent): _description_

        Returns:
            QueryRunEvent: sucessful execution of query , foraward to result inference
            QueryRunCorrectionEvent: correct faulty query by incorporating the error information
            ErrorEvent: incase of error
        """
        try:
            start_date = self.user_state.query_generator_response.start_date
            end_date = self.user_state.query_generator_response.end_date
            if 'now' in end_date:
                end_date = pd.Timestamp.now()
                start_date = end_date - pd.Timedelta(start_date.split('-')[1].split('/')[0])

            query = self.user_state.query_generator_response.query
            print('*'*50)
            print('Running the query')
            display(query)

            if len(query)>0:
                if start_date in ['', '*', None] and end_date in ['', '*', None]:
                    search_indexes = [self.opensearch_client.get_index(i.variable_name, self.city,None) 
                                    for i in self.user_state.schema_selector_response.indexes]
                else:
                    search_indexes = [self.opensearch_client.get_index(i.variable_name, self.city,date) 
                                    for date in pd.date_range(start_date, end_date, freq='MS') 
                                        for i in self.user_state.schema_selector_response.indexes]
                    
                print('going to fetch information from ', search_indexes)
                self.user_state.es_response = self.opensearch_client.get_data(query=query, opensearch_index=search_indexes)
                if len(self.user_state.es_response)==0:
                    self.user_state.error_lists.append('No ES data found')
                    self.logger_data_prepare.update_variables_on_rejection(stage='query run', 
                                                                   rejection_reasons=['No data found']
                                                                  )
                    return ErrorEvent(error_stage='query run')

                self.user_state.query_run_count = 0
                return QueryRunEvent()        
        except Exception as e:
            self.logger_data_prepare.update_variables_on_rejection(stage='query run', 
                                                                   rejection_reasons=[e.__str__]
                                                                  )
            self.handle_error('Query Runner', e)
            self.user_state.query_execution_error_lists.append(e.__str__)
            self.user_state.query_run_count += 1
            if self.user_state.query_run_count < self.max_errors:
                return QueryRunCorrectionEvent()
            return ErrorEvent()

    @step
    async def handle_query_validation_correction_event(self, ctx: Context,  ev:QueryValidationCorrectionEvent)->QueryGeneratorEvent|ErrorEvent:
        """
        Correct the symantic role of the query

        Args:
            ctx (Context): _description_
            ev (QueryValidationCorrectionEvent): _description_

        Returns:
            QueryGeneratorEvent: on successful query correction, send it to the query validator
            ErrorEvent: in case of error
        """
        try:
            # update the log with the rejection
            agent = self.agent_dict['query_corrector']
            agent_prompt = agent.agent_prompt.format(query_str=self.user_state.user_query,
                                                        schema_str=self.user_state.schema_selector_response.indexes,
                                                        input_str=QueryDetails.model_json_schema(),
                                                        es_query=self.user_state.query_generator_response.query,
                                                        error_list=self.user_state.validation_error_list
                                                            )
            response = self.run_agent(agent=agent, agent_prompt=agent_prompt)
            self.user_state.query_generator_response = response
            self.logger_data_prepare.initialize_query_state(data=response, stage='query validation correction')
            return QueryGeneratorEvent()        
        except Exception as e:
            self.handle_error('Query Validations Correction', e)
            return ErrorEvent(error_stage='query validation correction')
    
    @step
    async def handle_query_run_correction_event(self, ctx: Context,  ev:QueryRunCorrectionEvent)->QueryGeneratorEvent|ErrorEvent:
        """
        Correct query by incorporating its run time errors

        Args:
            ctx (Context): _description_
            ev (QueryRunCorrectionEvent): _description_

        Returns:
            QueryGeneratorEvent: send to the query validation for symantic validation
            ErrorEvent: on error
        """
        try:
            agent = self.agent_dict['query_corrector']
            agent_prompt = agent.agent_prompt.format(query_str=self.user_state.user_query,
                                                        schema_str=self.user_state.schema_selector_response.indexes,
                                                        input_str=QueryDetails.model_json_schema(),
                                                        es_query=self.user_state.query_generator_response.query,
                                                        error_list=self.user_state.query_execution_error_lists
                                                            )
            response = self.run_agent(agent=agent, agent_prompt=agent_prompt)
            self.user_state.query_generator_response = response
            self.logger_data_prepare.initialize_query_state(data=response, stage='query run correction')
            return QueryGeneratorEvent()        
        except Exception as e:
            self.handle_error('Query Run Correction', e)
            return ErrorEvent(error_stage='query run correction')

    @step
    async def handle_result_inference_event(self, ctx: Context,  ev:QueryRunEvent)->ResultInferenceEvent|ErrorEvent:
        """
        Prepare the inference data using LLM call with the es results
        Args:
            ctx (Context): _description_
            ev (QueryRunEvent): _description_

        Returns:
            ResultInferenceEvent: sucessful inference generation
            ErrorEvent: on error
        """
        try:
            self.logger_data_prepare.update_variables_on_sucess()
            query = self.user_state.query_generator_response.query
            agg_dict_bool = True if 'aggs' in query or 'aggs' in  query.get('query') else False
            query_dict_bool = True if 'query' in query else False

            results = self.user_state.es_response 

            if agg_dict_bool:
                results = results['aggregations']
            else:
                results = pd.json_normalize(pd.DataFrame(results.get('hits').get('hits'))['_source'])

            agent = self.agent_dict['response_synthesizer']
            agent_prompt = agent.agent_prompt.format(query_str=self.user_state.user_query,
                                                    schema_str=self.user_state.schema_selector_response.indexes,
                                                    results=results
                                                                )
            response = self.run_agent(agent=agent, agent_prompt=agent_prompt)
            self.user_state.query_response_synthesis = response
            return ResultInferenceEvent()
        except Exception as e:
            self.handle_error('Result Inference', e)
            return ErrorEvent(error_stage='result inference')

    @step
    async def handle_error_event(self, ctx: Context,  ev:ErrorEvent)->ErrorCompleteEvent:
        self.user_state.exited_stage = ev.error_stage
        self.user_state.exited_with_error = True
        return ErrorCompleteEvent()

    @step
    async def handle_stop_event(self, ctx: Context,  ev:ResultInferenceEvent|ErrorCompleteEvent)->StopEvent:
        mlflow.log_param('test_key',self.logger_data_prepare.query_list)
        self.logger.log_json_artifact(file_name='test_artifact', data=self.logger_data_prepare.query_list)
        self.logger.end_mlflow_logging()
        return StopEvent(self.user_state)


In [ ]:
# from llama_index.utils.workflow import draw_all_possible_flows
# workflow = ElastiSearchcAgent(agent_dict=None, llm='', schema_dict=None,
#                               opensearch_client=None, city=None)